# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          23 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/22 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.14927


extracting tarball to tmp_2210.14927...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.14931


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.14927/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'author_list' from 'tmp_2210.14927/author_list.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.15595


extracting tarball to tmp_2210.15595...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.15639


extracting tarball to tmp_2210.15639...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14927-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14927) | **Characterization Of Inpaint Residuals In Interferometric Measurements of  the Epoch Of Reionization**  |
|| Michael Pagano, et al. -- incl., <mark>Jing Liu</mark>, <mark>Adrian Liu</mark>, <mark>Jacqueline N. Hewitt</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *26 pages, 18 figures*|
|**Abstract**| Radio Frequency Interference (RFI) is one of the systematic challenges preventing 21cm interferometric instruments from detecting the Epoch of Reionization. To mitigate the effects of RFI on data analysis pipelines, numerous inpaint techniques have been developed to restore RFI corrupted data. We examine the qualitative and quantitative errors introduced into the visibilities and power spectrum due to inpainting. We perform our analysis on simulated data as well as real data from the Hydrogen Epoch of Reionization Array (HERA) Phase 1 upper limits. We also introduce a convolutional neural network that capable of inpainting RFI corrupted data in interferometric instruments. We train our network on simulated data and show that our network is capable at inpainting real data without requiring to be retrained. We find that techniques that incorporate high wavenumbers in delay space in their modeling are best suited for inpainting over narrowband RFI. We also show that with our fiducial parameters Discrete Prolate Spheroidal Sequences (DPSS) and CLEAN provide the best performance for intermittent ``narrowband'' RFI while Gaussian Progress Regression (GPR) and Least Squares Spectral Analysis (LSSA) provide the best performance for larger RFI gaps. However we caution that these qualitative conclusions are sensitive to the chosen hyperparameters of each inpainting technique. We find these results to be consistent in both simulated and real visibilities. We show that all inpainting techniques reliably reproduce foreground dominated modes in the power spectrum. Since the inpainting techniques should not be capable of reproducing noise realizations, we find that the largest errors occur in the noise dominated delay modes. We show that in the future, as the noise level of the data comes down, CLEAN and DPSS are most capable of reproducing the fine frequency structure in the visibilities of HERA data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14931-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14931) | **Cosmological inference from the EFTofLSS: the eBOSS QSO full-shape  analysis**  |
|| Théo Simon, <mark>Pierre Zhang</mark>, Vivian Poulin |
|*Appeared on*| *2022-10-28*|
|*Comments*| *33 + 13 pages, 8 figures. Comments welcome!*|
|**Abstract**| We present cosmological results inferred from the effective-field theory (EFT) analysis of the full-shape of eBOSS quasars (QSO) power spectrum. We validate our analysis pipeline against simulations, and find overall good agreement between the analyses in Fourier and configuration space. Keeping the baryon abundance and the spectral tilt fixed, we reconstruct at $68\%$ CL the fractional matter abundance $\Omega_m$, the reduced Hubble constant $h$, and the clustering amplitude $\sigma_8$, to respectively $\Omega_m=0.327\pm 0.035$, $h=0.655\pm 0.034$, and $\sigma_8=0.880\pm 0.083$ from eBOSS QSO alone. These constraints are consistent at $\lesssim 1.8\sigma$ with the ones from Planck and from the EFT analysis of BOSS full-shape. Interestingly $S_8$ reconstructed from eBOSS QSO is slightly higher than that deduced from Planck and BOSS, although statistically consistent. In combination with the EFT likelihood of BOSS, supernovae from Pantheon, and BAO from lyman-$\alpha$ and 6dF/MGS, constraints improve to $\Omega_m = 0.2985\pm 0.0069$ and $h = 0.6803\pm 0.0075$, in agreement with Planck and with similar precision. We also explore one-parameter extensions to $\Lambda$CDM and find that results are consistent with flat $\Lambda$CDM at $\lesssim 1.1\sigma$. We obtain competitive constraints on the curvature density fraction $\Omega_k=-0.039\pm 0.029$, the dark energy equation of state $w_0=-1.038\pm 0.041$, the effective number of relativistic species $N_{\rm eff}=3.44^{+0.44}_{-0.91}$ at $68\%$ CL, and the sum of neutrino masses $\sum m_\nu<0.274e$V at $95\%$ CL, without Planck data. Including Planck data, contraints significantly improve thanks to the large lever arm in redshift between LSS and CMB measurements. In particular, we obtain the stringent constraint $\sum m_\nu<0.093e$V, competitive with recent lyman-$\alpha$ forest power spectrum bound. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.14934-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.14934) | **The effect of outflows on CMB bounds from Primordial Black Hole  accretion**  |
|| Lorenzo Piga, et al. -- incl., <mark>Licia Verde</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *34 pages, 8 figures*|
|**Abstract**| Should Primordial Black Holes (PBHs) exist in nature, they would inevitably accrete baryonic matter in their vicinity. In turn, the consequent emission of high-energy radiation could affect the thermal history of the universe to an extent that can be probed with a number of cosmological observables such as the Cosmic Microwave Background (CMB) anisotropies. However, our understanding of the accretion and radiation emission processes in the context of PBHs is still in its infancy, and very large theoretical uncertainties affect the resulting constraints on the PBH abundance. Building on state-of-the-art literature, in this work we take a step towards the development of a more realistic picture of PBH accretion by accounting for the contribution of outflows. Specifically, we derive CMB-driven constraints on the PBH abundance for various accretion geometries, ionization models and mass distributions in absence and in presence of mechanical feedback and non-thermal emissions due to the outflows. As a result, we show that the presence of such outflows introduces an additional layer of uncertainty that needs to be taken into account when quoting cosmological constraints on the PBH abundance, with important consequences in particular in the LIGO-Virgo-KAGRA observational window. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15542-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15542) | **MHD decomposition explains diffuse $γ$-ray emission in Cygnus X**  |
|| Ottavio Fornieri, <mark>Heshou Zhang</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *7 pages, 4 figures*|
|**Abstract**| Cosmic-ray (CR) diffusion is the result of the interaction of such charged particles against magnetic fluctuations. These fluctuations originate from large-scale turbulence cascading towards smaller spatial scales, decomposed into three different modes, as described by $magneto-hydro-dynamics$ (MHD) theory. As a consequence, the description of particle diffusion strongly depends on the model describing the injected turbulence. Moreover, the amount of energy assigned to each of the three modes is in general not equally divided, which implies that diffusion properties might be different from one region to another. Here, motivated by the detection of different MHD modes inside the Cygnus-X star-forming region, we study the 3D transport of CRs injected by two prominent sources within a two-zone model that represents the distribution of the modes. Then, by convolving the propagated CR-distribution with the neutral gas, we are able to explain the $\gamma$-ray diffuse emission in the region, observed by the Fermi-LAT and HAWC Collaborations. Such a result represents an important step in the long-standing problem of connecting the CR observables with the micro-physics of particle transport. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15589-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15589) | **Kinematics of the Local Group gas and galaxies in the Hestia simulations**  |
|| Luis Biaus, et al. -- incl., <mark>Philipp Richter</mark>, <mark>Noam I. Libeskind</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *13 pages, 9 figures, accepted for publication in MNRAS*|
|**Abstract**| We investigate the kinematic properties of gas and galaxies in the Local Group (LG) using high-resolution simulations performed by the {\sc Hestia} (High-resolution Environmental Simulations of The Immediate Area) collaboration. Our simulations include the correct cosmography surrounding LG-like regions consisting of two main spiral galaxies of $\sim 10^{12}$~M$_\odot$, their satellites and minor isolated galaxies, all sharing the same large-scale motion within a volume of a few Mpc. We characterise the gas and galaxy kinematics within the simulated LGs, from the perspective of the Sun, to compare with observed trends from recent HST/COS absorption-line observations and LG galaxy data. To analyse the velocity pattern of LG gas and galaxies seen in the observational data, we build sky maps from the local standard of rest, and the galactic and local group barycentre frames. Our findings show that the establishment of a radial velocity dipole at low/high latitudes, near the preferred barycentre direction, is a natural outcome of simulation kinematics for material {\it outside} the Milky Way virial radius after removing galaxy rotation when the two main LG galaxies are approaching. Our results favour a scenario where gas and galaxies stream towards the LG barycentre producing a velocity dipole resembling observations. While our study shows in a qualitative way the global matter kinematics in the LG as part of its on-going assembly, quantitative estimates of gas-flow rates and physical conditions of the LG gas have to await a more detailed modeling of the ionization conditions, which will be presented in a follow-up paper. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15595-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15595) | **A deep 1.4 GHz survey of the J1030 equatorial field: a new window on  radio source populations across cosmic time**  |
|| Q. D'Amato, et al. -- incl., <mark>E. Liuzzo</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *18 pages, 15 figures, 5 table. Accepted for publication in A&A*|
|**Abstract**| We present deep L-Band observations of the equatorial field centered on the z=6.3 SDSS QSO, reaching a 1 sigma sensitivity of ~2.5 uJy at the center of the field. We extracted a catalog of 1489 radio sources down to a flux density of ~12.5 uJy (5 sigma) over a field of view of ~ 30' diameter. We derived the source counts accounting for catalog reliability and completeness, and compared them with others available in the literature. Our source counts are among the deepest available so far, and, overall, are consistent with recent counts' determinations and models. We detected for the first time in the radio band the SDSS J1030+0524 QSO (26 +/- 5 uJy). We derived its optical radio loudness R_O = 0.62 +/- 0.12, which makes it the most radio quiet AGN at z >~ 6 discovered so far and detected at radio wavelengths. We unveiled extended diffuse radio emission associated with the lobes of a bright FRII radio galaxy located close to the center of the J1030 field, which is likely to become the future BCG of a protocluster at z=1.7. The lobes' complex morphology, coupled with the presence of X-ray diffuse emission detected around the FRII galaxy lobes, may point toward an interaction between the radio jets and the external medium. We also investigated the relation between radio and X-ray luminosity for a sample of 243 X-ray-selected objects obtained from 500 ks Chandra observations of the same field, and spanning a wide redshift range (0 ~< z ~< 3). Focused on sources with a spectroscopic redshift and classification, we found that sources hosted by ETG and AGN follow Log(L_R)/Log(L_X) linear correlations with slopes of ~0.6 and ~0.8, respectively. This is interpreted as a likely signature of different efficiency in the accretion process. Finally, we found that most of these sources (>~87%) show a radio-to-X-ray radio loudness R_X < -3.5, classifying these objects as radio quiet. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15639-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15639) | **A shot in the Dark (Ages): a faint galaxy at $z=9.76$ confirmed with  JWST**  |
|| Guido Roberts-Borsani, et al. -- incl., <mark>Nicha Leethochawalit</mark>, <mark>Lifan Wang</mark>, <mark>Lilan Yang</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *Submitted to Nature. 34 pages, 4 main figures, 1 supplementary figure, 2 supplementary tables. Comments are welcome*|
|**Abstract**| The appearance of galaxies over the first billion years after the Big Bang is believed to be responsible for the last dramatic change in the state of the Universe. Ultraviolet photons from galaxies within this time period - the Epoch of Reionization - ionized intergalactic Hydrogen, rendering the Universe transparent to UV radiation and ending the so-called cosmic Dark Ages, sometime after redshift $z\sim8$. The majority of ionizing photons in the first few hundred Myrs of cosmic history are thought to derive from galaxies significantly fainter than the characteristic luminosity $L^{*}$. These faint galaxies are thought to be surrounded by sufficient neutral gas to prevent the escape of the Lyman-$\alpha$ photons that would allow confirmation with current observatories. Here we demonstrate the power of the recently commissioned James Webb Space Telescope to transform our understanding of the sources of reionization, by reporting the first spectroscopic confirmation of a very low luminosity ($\sim0.05 L^{*}$) galaxy at $z=9.76$, observed 480 Myr after the Big Bang, via the detection of the Lyman-break and redward continuum with the NIRSpec and NIRCam instruments. The galaxy JD1 is gravitationally magnified by a factor of $\mu\sim13$ by the foreground cluster A2744. The power of JWST and lensing allows us to peer deeper than ever before into the cosmic Dark Ages, revealing the compact ($\sim$150 pc) and complex morphology and physical properties of an ultrafaint galaxy ($M_{\rm UV}=-17.45$). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.15525-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.15525) | **The Gaia-ESO survey: mapping the shape and evolution of the radial  abundance gradients with open clusters**  |
|| L. Magrini, et al. -- incl., <mark>U. Heiter</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2022-10-28*|
|*Comments*| *25 pages, 14 figures and 4 tables in the main text, 3 figures and 7 tables in the Appendix. Accepted for publication in A&A*|
|**Abstract**| The spatial distribution of elemental abundances and their time evolution are among the major constraints to disentangle the scenarios of formation and evolution of the Galaxy. We used the sample of open clusters available in the final release of the Gaia-ESO survey to trace the Galactic radial abundance and abundance to iron ratio gradients, and their time evolution. We selected member stars in 62 open clusters, with ages from 0.1 to about 7~Gyr, located in the Galactic thin disc at Galactocentric radii from about 6 to 21~kpc. We analysed the shape of the resulting [Fe/H] gradient, the average gradients [El/H] and [El/Fe] combining elements belonging to four different nucleosynthesis channels, and their individual abundance and abundance ratio gradients. We also investigated the time evolution of the gradients dividing open clusters in three age bins. The[Fe/H] gradient has a slope of -0.054 dex~kpc-1. We saw different behaviours for elements belonging to different channels. We found that the youngest clusters in the inner disc have lower metallicity than their older counterpart and they outline a flatter gradient. We considered some possible explanations, including the effects of gas inflow and migration. We suggested that it might be a bias introduced by the standard spectroscopic analysis producing lower metallicities in low gravity stars. To delineate the shape of the `true' gradient, we should limit our analysis to stars with low surface gravity logg>2.5 and xi<1.8 km~s-1. Based on this reduced sample, we can conclude that the gradient has minimally evolved over the time-frame outlined by the open clusters, indicating a slow and stationary formation of the thin disc in the latest Gyr. We found a secondary role of clusters' migration in shaping the gradient, with a more prominent role of migration for the oldest clusters. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure FIGURE/USED/62RgcXHXFEbyagelowessmeanv13.pdf</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

73  publications in the last 7 days.
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers